In [1]:
# Import libraries
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold, cross_val_score, KFold
import tensorflow as tf
from keras.layers import Dense, Input

In [2]:
# Read the data
df = pd.read_csv('input/train.csv')
test = pd.read_csv('input/test.csv')

# get X, y from the training data
X = df.drop(['id', 'defects'], axis=1).values
y = df.defects.astype('int').values

In [3]:
# Use sklearn KFold cross validation
stratified_cv = StratifiedKFold(n_splits=10)

In [4]:
# Apply cross_val_score on the created model using metric "roc_auc"

n_splits = 5
kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
accuracies = []
for train_index, val_index in kf.split(X, y):
    X_fold, X_val_fold = X[train_index], X[val_index]
    y_fold, y_val_fold = y[train_index], y[val_index]

    # Create and train your neural network model
    model = tf.keras.Sequential([
        Input(shape=(X_fold.shape[1],)),  # Input layer with input_dim features
        Dense(8, activation='relu'),
        Dense(4, activation='relu'),
        Dense(1, activation='sigmoid')  # Output layer with sigmoid activation to get output in range [0, 1]
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy')  # Adjust loss and optimizer as needed
    history = model.fit(X_fold, y_fold, epochs=50, batch_size=32,
                    validation_data=(X_val_fold, y_val_fold),
                    callbacks=[tf.keras.callbacks.EarlyStopping(patience=10)])

    # Evaluate the model on the validation data
    y_pred = model.predict(X_val_fold)
    accuracy = roc_auc_score(y_val_fold, y_pred)
    accuracies.append(accuracy)
    print(f'Accuracy on validation fold: {accuracy}')

# After iterating through all folds, analyze the results
print(accuracies)

Epoch 1/50
2545/2545 [==============================] - 3s 1ms/step - loss: 61.4897 - val_loss: 1.8505
Epoch 2/50
2545/2545 [==============================] - 3s 1ms/step - loss: 0.5384 - val_loss: 0.5235
Epoch 3/50
2545/2545 [==============================] - 3s 1ms/step - loss: 0.5496 - val_loss: 0.5347
Epoch 4/50
2545/2545 [==============================] - 3s 1ms/step - loss: 0.5406 - val_loss: 0.5352
Epoch 5/50
2545/2545 [==============================] - 3s 992us/step - loss: 0.5358 - val_loss: 0.5210
Epoch 6/50
2545/2545 [==============================] - 3s 1ms/step - loss: 0.5123 - val_loss: 0.4960
Epoch 7/50
2545/2545 [==============================] - 3s 1ms/step - loss: 0.5078 - val_loss: 0.4871
Epoch 8/50
2545/2545 [==============================] - 3s 1ms/step - loss: 0.4954 - val_loss: 0.5133
Epoch 9/50
2545/2545 [==============================] - 3s 1ms/step - loss: 0.4893 - val_loss: 0.4752
Epoch 10/50
2545/2545 [==============================] - 3s 1ms/step - loss: 0.

In [13]:
accuracies
np.mean(accuracies)

0.7176194898533594

In [5]:
# Try different Networks and see how the score will change

In [9]:
for train_index, val_index in kf.split(X, y):
    print(len(train_index), len(val_index))

81410 20353
81410 20353
81410 20353
81411 20352
81411 20352
